# مرحله ۱: نصب تمام کتابخانه‌های مورد نیاز

wandb : e3b067db7cb39378cff354e9738c4095f9dbb0d5


In [1]:
print("⏳ Starting to install libraries...")
!pip install "unsloth[colab-new]" -q
!pip install --no-deps xformers -q
!pip install "transformers[torch]" "datasets" "accelerate" "bitsandbytes" -q
!pip install sentence-transformers -q
!pip install lancedb -q
!pip install python-docx -q
print("✅ The libraries were installed successfully.")

⏳ Starting to install libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.8/184.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.4 MB/s eta 0:00:00
✅ The libraries were installed successfully.


# مرحله ۲: ایمپورت کتابخانه‌ها و تعریف توابع اولیه


---

### توضیح کد

در این بخش دو تابع تعریف شده‌اند:

#### 1. تابع `read_docx(file_path)`

این تابع مسئول خواندن محتوای یک فایل **Word با فرمت docx** است.

* ابتدا تلاش می‌کند با استفاده از کتابخانه‌ی `python-docx` فایل را باز کند:

  ```python
  doc = docx.Document(file_path)
  ```
* سپس تمام پاراگراف‌های موجود در فایل را می‌خواند و متن آن‌ها را در یک لیست ذخیره می‌کند:

  ```python
  full_text = [para.text for para in doc.paragraphs]
  ```
* در نهایت، همه‌ی پاراگراف‌ها را با یک **خط جدید (`\n`)** به هم وصل کرده و خروجی به صورت یک رشته‌ی کامل برمی‌گرداند:

  ```python
  return '\n'.join(full_text)
  ```
* اگر خطایی رخ دهد (مثلاً مسیر فایل اشتباه باشد یا فایل خراب باشد)، پیام خطا چاپ شده و مقدار `None` بازگردانده می‌شود.

---

#### 2. تابع `preprocess_persian(text)`

این تابع برای **پاک‌سازی و استانداردسازی متن فارسی** طراحی شده است.

* ابتدا حروف عربی **(ك و ي)** را به معادل فارسی‌شان **(ک و ی)** تبدیل می‌کند تا یکدست شوند:

  ```python
  text = text.replace('ك', 'ک').replace('ي', 'ی')
  ```
* سپس با استفاده از `regex` تمام کاراکترهای غیرمجاز (هر چیزی به جز حروف، اعداد، فاصله، نقطه و ویرگول فارسی "،") را حذف می‌کند:

  ```python
  text = re.sub(r'[^\w\s.،]', '', text)
  ```
* بعد، فاصله‌های اضافه و پشت سر هم به یک فاصله‌ی ساده تبدیل می‌شوند:

  ```python
  text = re.sub(r'\s+', ' ', text)
  ```
* در پایان، متن تمیز و مرتب شده بازگردانده می‌شود.

---



In [ ]:
import docx
import re
import random
import numpy as np
import pandas as pd
import lancedb
import torch
from sentence_transformers import SentenceTransformer, losses, InputExample
from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print("The libraries were successfully imported. 📚")

def read_docx(file_path):
    try:
        doc = docx.Document(file_path)
        full_text = [para.text for para in doc.paragraphs]
        return '\n'.join(full_text)
    except Exception as e:
        print(f"Error to read file!{e}")
        return None

def preprocess_persian(text):
    text = text.replace('ك', 'ک').replace('ي', 'ی')
    text = re.sub(r'[^\w\s.،]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# مرحله ۳: بارگذاری و پیش‌پردازش داده‌ها

#### 1. بخش‌بندی متن (Chunking)

برای بخش‌بندی، از **عبارت منظم (Regex)** استفاده شده است:

```python
chunks = re.split(r'\n\d+\s*\.', document_text)
```

این عبارت به دنبال الگوهایی مانند **شماره + نقطه** (مثل "1." یا "2.") در ابتدای خط می‌گردد و متن را بر اساس آن به بخش‌های جدا تقسیم می‌کند. سپس با این دستور، بخش‌های خالی حذف و فاصله‌های اضافی حذف می‌شوند:

```python
chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
```

---

#### 2. استخراج عناوین (Titles)

عناوین بخش‌ها که معمولاً به صورت **شماره + متن** هستند، با دستور زیر از متن گرفته می‌شوند:

```python
titles = re.findall(r'(\d+\s*\..+)', document_text)
```

در ادامه بررسی می‌شود که تعداد عنوان‌ها و بخش‌ها با هم هماهنگ باشند. اگر بخش‌های بیشتری از عنوان‌ها وجود داشت، فقط به تعداد مشترک بین آن‌ها ترکیب صورت می‌گیرد:

```python
num_chunks_to_title = min(len(titles), len(chunks) - 1)
for i in range(num_chunks_to_title):
    chunks[i+1] = titles[i] + '\n' + chunks[i+1]
```

در این مرحله، هر بخش با عنوان خودش همراه می‌شود تا ساختار متن حفظ گردد.

---

#### 3. پیش‌پردازش متن

در پایان، تمام بخش‌ها (chunks) با استفاده از تابع `preprocess_persian` پردازش و پاک‌سازی می‌شوند. نتیجه در لیست `processed_chunks` ذخیره می‌شود:

```python
processed_chunks = [preprocess_persian(chunk) for chunk in chunks]
```

---


In [3]:
file_path = 'Guilan-Food.docx'
document_text = read_docx(file_path)

if document_text:
    # بخش‌بندی متن (Chunking)
    chunks = re.split(r'\n\d+\s*\.', document_text)
    chunks = [chunk.strip() for chunk in chunks if chunk.strip()]
    titles = re.findall(r'(\d+\s*\..+)', document_text)
    if len(chunks) > 1 and len(titles) >= len(chunks) - 1:
        # To handle potential empty splits, we match titles to available chunks
        num_chunks_to_title = min(len(titles), len(chunks) - 1)
        for i in range(num_chunks_to_title):
            chunks[i+1] = titles[i] + '\n' + chunks[i+1]

    # پیش‌پردازش نهایی
    processed_chunks = [preprocess_persian(chunk) for chunk in chunks]
    print(f"\n✅ The data were successfully split into {len(processed_chunks)} chunks and preprocessed.")
else:
    print("❌ Processing stopped because the file could not be read.")


✅ The data were successfully split into 14 chunks and preprocessed.


# مدل‌های امبدینگ مرحله Fine-Tuning مرحله 4: ایجاد دیتاست مصنوعی و

در این بخش فرآیند **فاین‌تیون (Fine-Tuning) مدل‌های امبدینگ** برای داده‌های پرسش‌وپاسخ شروع می‌شود.

ابتدا یک دیتاست مصنوعی از چند **پرسش مرتبط با غذاهای گیلانی** ساخته شده است. در این دیتاست، برای هر بخش از متن (که قبلاً در مرحله‌ی پردازش به صورت chunk آماده شده بود) چند پرسش متنی در نظر گرفته می‌شود. به عنوان مثال، برای بخش مربوط به "باقلا قاتوق"، پرسش‌هایی مثل «چطور درست می‌شود؟» یا «مواد لازم چیست؟» ثبت شده‌اند.

سپس، از بین بخش‌های متن، برای هر پرسش یک **پاسخ مثبت** (بخش مرتبط) و یک **پاسخ منفی** (بخش نامرتبط به صورت تصادفی) انتخاب می‌شود. این کار باعث می‌شود مدل در حین آموزش بتواند یاد بگیرد که کدام متن به پرسش مرتبط است و کدام متن نامربوط است. همه‌ی این ترکیب‌ها به صورت نمونه‌های آموزشی در لیستی به نام `train_examples` ذخیره می‌شوند.

در ادامه، چند مدل مختلف برای آزمایش انتخاب شده‌اند، شامل:

* مدل **LaBSE** برای چندزبانه‌ها
* مدل **multilingual-e5-base** بهینه‌شده برای جست‌وجوی معنایی
* مدل **bert-fa-base-uncased** مخصوص زبان فارسی

برای هر مدل، مراحل آموزش به این صورت انجام می‌شود:

1. مدل بارگذاری می‌شود.
2. یک تابع هزینه به نام **MultipleNegativesRankingLoss** تعریف می‌شود که مناسب یادگیری تفاوت بین پاسخ‌های درست و نادرست است.
3. داده‌های آموزشی در قالب مینی‌بچ (batch) آماده می‌شوند.
4. مدل طی چند epoch روی این داده‌ها آموزش می‌بیند.

در نهایت، نسخه‌ی فاین‌تیون‌شده‌ی هر مدل در دیکشنری `tuned_models` ذخیره می‌شود. وقتی آموزش همه‌ی مدل‌ها تمام شود، پیامی مبنی بر موفقیت فرآیند فاین‌تیون چاپ خواهد شد.

---

🔹 به طور خلاصه:

* ابتدا داده‌های پرسش‌وپاسخ ساخته شدند.
* سپس برای هر پرسش، متن مرتبط و نامرتبط انتخاب شد.
* سه مدل امبدینگ روی این داده‌ها آموزش داده شدند.
* نتیجه، مدل‌های فاین‌تیون‌شده‌ای هستند که برای جست‌وجو و بازیابی پرسش‌وپاسخ دقیق‌تر عمل می‌کنند.

---

In [4]:
print("\n⏳ Starting the embedding models fine-tuning process...")
# دیتاست مصنوعی پرسش و پاسخ
qa_pairs = {
    0: ["باقلا قاتوق چطور درست میشه؟", "مواد لازم برای باقلا قاتوق چیست؟"],
    1: ["طرز تهیه میرزا قاسمی رو توضیح بده.", "برای طعم دودی میرزا قاسمی چیکار باید کرد؟"],
    2: ["راز کباب ترش گیلانی چیه؟", "چه سبزی‌هایی در کباب ترش استفاده میشه؟"],
    10: ["خورشت چغرتمه با چی درست میشه؟", "نقش تخم مرغ در چغرتمه چیه؟"],
    13: ["سیرابیج چیه؟", "غذای سیرابیج چطور آماده می‌شود؟"]
}

train_examples = []
chunk_indices = list(range(len(processed_chunks)))
# Ensure all chunk_ids in qa_pairs are valid
valid_qa_pairs = {k: v for k, v in qa_pairs.items() if k < len(processed_chunks)}

for chunk_id, questions in valid_qa_pairs.items():
    positive_passage = processed_chunks[chunk_id]
    for question in questions:
        negative_chunk_id = random.choice([i for i in chunk_indices if i != chunk_id])
        negative_passage = processed_chunks[negative_chunk_id]
        train_examples.append(InputExample(texts=[question, positive_passage, negative_passage]))

# لیست مدل‌ها برای آموزش
model_names = [
    'sentence-transformers/LaBSE',
    'intfloat/multilingual-e5-base',
    'HooshvareLab/bert-fa-base-uncased'
]
tuned_models = {}

for model_name in model_names:
    print(f"\n----- Training model: {model_name} -----")
    model = SentenceTransformer(model_name)
    train_loss = losses.MultipleNegativesRankingLoss(model)
    train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=2, show_progress_bar=True)
    tuned_models[model_name] = model
print("\n✅ The Fine-Tuning process completed successfully.")


⏳ Starting the embedding models fine-tuning process...

----- Training model: sentence-transformers/LaBSE -----


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss



✅ The Fine-Tuning process completed successfully.


# مرحله ۵: ساخت پایگاه‌داده‌ برداری

---

در این بخش، برای هر مدل فاین‌تیون‌شده یک **پایگاه داده برداری (Vector Database)** ساخته می‌شود. هدف از این کار، ذخیره‌سازی متن‌ها به همراه بردارهای امبدینگ آن‌هاست تا در مراحل بعدی، جست‌وجوی معنایی سریع و دقیق انجام گیرد.

ابتدا مدل‌های فاین‌تیون‌شده یکی‌یکی پردازش می‌شوند. برای هر مدل، متن‌های پردازش‌شده (chunks) به بردارهای عددی تبدیل می‌شوند. این بردارها نمایانگر معنای جملات هستند و توسط خود مدل تولید می‌شوند.

سپس یک پایگاه داده جداگانه برای هر مدل ایجاد می‌گردد. مسیر ذخیره‌سازی دیتابیس به صورت موقت در پوشه `/tmp` تعریف شده و اگر قبلاً دیتابیس مشابهی وجود داشته باشد، پاک می‌شود تا یک دیتابیس جدید ساخته شود.

هر بخش از متن همراه با بردار مربوطه و یک شناسه (id) در قالب یک جدول به نام **guilan\_food** ذخیره می‌شود. این جدول شامل سه ستون است:

* **vector** → بردار امبدینگ متن
* **text** → متن بخش مربوطه
* **id** → شماره یا شناسه‌ی هر بخش

در پایان، اطلاعات هر مدل (شامل جدول و بردارها) در دیکشنری `vector_dbs` ذخیره می‌شود. این ساختار باعث می‌شود بتوان به راحتی به پایگاه داده برداری مربوط به هر مدل دسترسی داشت.

---

In [5]:
print("\n⏳ Building the vector database (LanceDB)...")
vector_dbs = {}
for model_name, model in tuned_models.items():
    print(f"----- Processing for model: {model_name} -----")
    embeddings = model.encode(processed_chunks, show_progress_bar=True)

    # LanceDB
    db_path = f"/tmp/lancedb_{model_name.replace('/', '_')}"
    !rm -rf {db_path}
    db = lancedb.connect(db_path)
    data = [{"vector": emb.tolist(), "text": chunk, "id": i} for i, (emb, chunk) in enumerate(zip(embeddings, processed_chunks))]
    tbl = db.create_table("guilan_food", data=data, mode="overwrite")

    vector_dbs[model_name] = {
        "lancedb_table": tbl,
        "embeddings": embeddings
    }
print("\n✅ The vector database was created successfully.")


⏳ Building the vector database (LanceDB)...
----- Processing for model: sentence-transformers/LaBSE -----


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

----- Processing for model: intfloat/multilingual-e5-base -----


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

----- Processing for model: HooshvareLab/bert-fa-base-uncased -----


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


✅ The vector database was created successfully.


# مرحله ۶: ارزیابی عملکرد سیستم بازیابی

---

در این قسمت، سیستم بازیابی (Retrieval System) ارزیابی می‌شود تا مشخص شود که مدل‌ها تا چه حد می‌توانند پرسش‌های تعریف‌شده را با بخش درست از متن تطبیق دهند.

ابتدا یک دیتاست ارزیابی ساخته می‌شود. در این دیتاست، برای هر پرسش ثبت می‌شود که پاسخ صحیح آن در کدام بخش از متن (chunk) قرار دارد. این کار باعث می‌شود که در هنگام ارزیابی بتوانیم صحت نتایج بازیابی‌شده را بررسی کنیم.

سپس، برای هر پرسش، بردار امبدینگ پرسش تولید شده و در پایگاه داده برداری جست‌وجو انجام می‌گیرد. نتایج بازیابی شامل چند بخش از متن است که بیشترین شباهت را به پرسش دارند. برای هر نتیجه، سه معیار اصلی محاسبه می‌شود:

1. **Hit\@k** → بررسی می‌کند آیا بخش درست (true\_id) در بین k نتیجه‌ی اول وجود دارد یا نه.
2. **MRR (Mean Reciprocal Rank)** → جایگاه اولین پاسخ درست در لیست نتایج را اندازه‌گیری می‌کند. هرچه پاسخ درست بالاتر باشد، مقدار MRR بیشتر خواهد بود.
3. **Average Cosine Similarity** → میزان شباهت میانگین بین پرسش و اولین نتیجه بازیابی‌شده را محاسبه می‌کند. این معیار نشان می‌دهد که پرسش و متن بازیابی‌شده از نظر معنایی چقدر به هم نزدیک هستند.

در پایان، برای هر مدل فاین‌تیون‌شده، مقادیر این معیارها محاسبه و در قالب یک جدول (DataFrame) نمایش داده می‌شود. این جدول شامل نام مدل، نوع پایگاه داده (اینجا LanceDB) و مقادیر سه معیار فوق است.

---


In [6]:
print("\n⏳ Starting the recovery system assessment...")
# دیتاست ارزیابی
evaluation_dataset = []
for chunk_id, questions in valid_qa_pairs.items():
    for question in questions:
        evaluation_dataset.append({"question": question, "true_id": chunk_id})


def calculate_metrics(retrieved_ids, true_id, k):
    retrieved_at_k = retrieved_ids[:k]
    is_hit = int(true_id in retrieved_at_k)
    rr = 1.0 / (retrieved_at_k.index(true_id) + 1) if is_hit else 0.0
    return {f"hit@{k}": is_hit, "rr": rr}


def evaluate_retrieval(model_name, k=5):
    model = tuned_models[model_name]
    db_info = vector_dbs[model_name]
    total_metrics = {f"hit@{k}": 0, "mrr": 0, "avg_cosine_sim": 0}

    for item in evaluation_dataset:
        query_embedding = model.encode([item["question"]])
        results = db_info["lancedb_table"].search(query_embedding[0]).limit(k).to_list()
        retrieved_ids = [res['id'] for res in results]

        if not retrieved_ids:
            continue  # Skip if no results

        retrieved_embedding = db_info["embeddings"][retrieved_ids[0]]
        total_metrics["avg_cosine_sim"] += cosine_similarity(query_embedding, [retrieved_embedding])[0][0]

        metrics = calculate_metrics(retrieved_ids, item["true_id"], k)
        total_metrics[f"hit@{k}"] += metrics[f"hit@{k}"]
        total_metrics["mrr"] += metrics["rr"]

    num_queries = len(evaluation_dataset)
    return {key: val / num_queries for key, val in total_metrics.items()} if num_queries > 0 else total_metrics


# اجرای ارزیابی
results = []
K = 5
for model_name in tuned_models.keys():
    metrics = evaluate_retrieval(model_name, k=K)
    metrics['model'] = model_name
    metrics['db'] = 'lancedb'
    results.append(metrics)

df_results = pd.DataFrame(results)[['model', 'db', f'hit@{K}', 'mrr', 'avg_cosine_sim']]
print("\n----- 📊 Retrieval System Evaluation Results -----")
print(df_results.to_string())
print("✅ The evaluation was completed successfully.")


⏳ Starting the recovery system assessment...

----- 📊 Retrieval System Evaluation Results -----
                               model       db  hit@5       mrr  avg_cosine_sim
0        sentence-transformers/LaBSE  lancedb    0.1  0.025000        0.310347
1      intfloat/multilingual-e5-base  lancedb    0.6  0.333333        0.830152
2  HooshvareLab/bert-fa-base-uncased  lancedb    0.2  0.045000        0.621583
✅ The evaluation was completed successfully.


# Rag مرحله ۷: راه‌اندازی مدل تولیدکننده و ساخت سیستم

---

در این بخش، یک **مدل زبانی بزرگ (LLM)** بارگذاری می‌شود تا بر اساس متونی که از سیستم بازیابی به دست می‌آید، پاسخ پرسش‌ها تولید کند.

ابتدا بهترین مدل بازیابی انتخاب می‌شود. این کار با توجه به معیار **MRR** انجام می‌شود، چون این معیار نشان می‌دهد که مدل تا چه حد توانسته پاسخ درست را در رتبه‌های بالای نتایج قرار دهد. مدلی که بالاترین مقدار MRR را داشته باشد، به عنوان بهترین سیستم انتخاب می‌شود.

سپس مدل **Llama-3 (نسخه 8 میلیارد پارامتر)** با استفاده از کتابخانه‌ی **unsloth** بارگذاری می‌شود. این نسخه به صورت فشرده‌سازی‌شده در حالت **4bit** روی کارت گرافیک اجرا می‌شود تا روی Google Colab Free هم قابل استفاده باشد. اگر بارگذاری موفقیت‌آمیز باشد، پیام تأیید چاپ می‌شود و در غیر این صورت، پیام خطا نمایش داده خواهد شد.

پس از بارگذاری مدل، تابعی به نام `answer_question` تعریف شده است. این تابع وظیفه دارد به هر پرسش کاربر پاسخ دهد. روند کار به این صورت است:

1. **بازیابی (Retrieval):**
   پرسش کاربر به بردار امبدینگ تبدیل می‌شود. سپس این بردار در پایگاه داده برداری جست‌وجو می‌شود و سه بخش از متن که بیشترین شباهت را دارند، به عنوان زمینه (context) انتخاب می‌شوند.

2. **تولید پاسخ (Generation):**
   یک **پرامپت (prompt)** ساخته می‌شود که شامل متن زمینه و پرسش کاربر است. در این پرامپت به مدل گفته می‌شود که فقط پاسخ مستقیم را به زبان فارسی بدهد و از تکرار پرسش یا توضیح اضافه خودداری کند.

   سپس این پرامپت به مدل Llama-3 داده می‌شود و خروجی آن پاسخ پرسش خواهد بود. در نهایت، فقط بخش مربوط به "Answer" استخراج و به کاربر نمایش داده می‌شود.

---

In [7]:
print("\n⏳ Loading Large Language Model (LLM) to generate answers...")
# انتخاب بهترین سیستم بازیابی بر اساس MRR
best_setup = df_results.sort_values(by='mrr', ascending=False).iloc[0]
best_model_name = best_setup['model']
print(f"🚀 Best recovery model: '{best_model_name}' with LanceDB database")

# بارگذاری Llama-3 با unsloth
try:
    model_id = "unsloth/llama-3-8b-bnb-4bit"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model_gen = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        load_in_4bit=True,
    )
    text_generator = pipeline("text-generation", model=model_gen, tokenizer=tokenizer)
    print("✅ The Llama-3 model was successfully loaded.")
except Exception as e:
    print(f"❌ Error loading Llama-3 model: {e}")
    text_generator = None


def answer_question(query):
    if not text_generator:
        return "The manufacturer model is not loaded."

    # 1. بازیابی (Retrieval)
    retrieval_model = tuned_models[best_model_name]
    db_info = vector_dbs[best_model_name]
    query_embedding = retrieval_model.encode([query])

    results = db_info["lancedb_table"].search(query_embedding[0]).limit(3).to_list()
    context = "\n---\n".join([res['text'] for res in results])

    # 2. تولید (Generation)
    prompt = f"""Based on the following context, only answer the question in Persian.
    Do not repeat the question or give explanations.
    Do not ask the question.
    Only return the direct answer.

    Context: {context}

    Question: {query}

    Answer:"""
    # Set a pad token if it's not already set
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    outputs = text_generator(prompt, max_new_tokens=300, pad_token_id=tokenizer.eos_token_id, temperature=0.7)
    return outputs[0]['generated_text'].split("Answer:")[-1].strip()


⏳ Loading Large Language Model (LLM) to generate answers...
🚀 Best recovery model: 'intfloat/multilingual-e5-base' with LanceDB database


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:231: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

Device set to use cuda:0


✅ The Llama-3 model was successfully loaded.


 # مرحله ۸: تست نهایی سیستم

In [8]:
print("----- 🤖 Final test of the RAG system -----")
test_query = "برای کباب ترش از چه گوشتی استفاده می‌شود و چطور مزه‌دار می‌شود؟"
final_answer = answer_question(test_query)

print(f"❓ سوال: {test_query}")
print(f"\n💬 پاسخ تولید شده:\n{final_answer}")



----- 🤖 Final test of the RAG system -----
❓ سوال: برای کباب ترش از چه گوشتی استفاده می‌شود و چطور مزه‌دار می‌شود؟

💬 پاسخ تولید شده:
ترش کباب از گوشت گوسفند درست می‌شود و با آبغوره، آب انار یا آب نارنج و رب انار مزه‌دار می‌شود.
